In [29]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!pip install Textblob

In [31]:
from textblob import TextBlob

In [32]:
import pandas as pd

In [33]:
data_path = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/ProsessedSeabank.csv'

In [34]:
!pip3 install googletrans==3.1.0a0

In [35]:
# Importing library yang dibutuhkan
import googletrans
from googletrans import Translator

In [36]:
data = pd.read_csv(data_path)
data.head()

,content_normalisasi2
0,bagus
1,mantap
2,gk saldo minimal
3,oke
4,urgent


In [37]:
translator = Translator()
translations = {}
for column in data.columns:
    # Unique elements of the column
    unique_elements = data[column].unique()
    for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
        translations[element] = translator.translate(element).text
translations

{'bagus': 'Good',
 'mantap': 'solid',
 'gk saldo minimal': 'no minimum balance',
 'oke': 'limit',
 'urgent': 'urgent',
 'terima kasih seabank terbaik': 'thank you best seabank',
 'bayar qris kena x bayar berhasil gagal dana yg terpotong x email c memuaskan udah gak solusi c nya': 'pay qris hit x pay successful fail funds deducted x email c is satisfactory no longer is the solution c',
 'seabank okcepat': 'seabank okcepat',
 nan: 'in',
 'nyaman mudah transfer tarik tunai': 'convenient easy transfer cash withdrawal',
 'gak ribet transfer kemna aja seabank emang bagus mntap': 'No hassle, transfer anywhere, Seabank is really good',
 'kali seabank aktifkan seabank pinjam th loh pake seabank th pake shopee ga telat bayar': "Seabank activates seabank loans, you use Seabank, you don't pay late using Shopee",
 'no coment': 'no comment',
 'baguss banget': 'very good',
 'baguss mudah': 'Nice easy',
 'seabank memank mantaappzz': 'Jayabank Meemonk Mantavj',
 'memudahkan transaksi': 'make transactio

In [38]:
data.replace(translations, inplace = True)
data

,content_normalisasi2
0,Good
1,solid
2,no minimum balance
3,limit
4,urgent
...,...
3439,Thank you Seabank
3440,Satisfying free transfers make it easier for u...
3441,simple easy fast
3442,"Enter my account, the balance is sufficient, I..."


In [39]:
data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/Seabank_DataTranslated.xlsx', index=False)

In [40]:
#SPLIT DATA
import pandas as pd

dataset = pd.read_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/Seabank_DataTranslated.xlsx')

# Hitung jumlah baris dalam dataset
num_rows = len(dataset)

# Hitung jumlah baris yang akan digunakan sebagai pelatihan (disini 50%)
train_size = int(0.5 * num_rows)

# Bagi dataset menjadi subset pelatihan dan pengujian
train_data = dataset[:train_size]
test_data = dataset[train_size:]

# Simpan subset pelatihan ke dalam file xlsx
train_data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/Seabank_DataTrainEnglish.xlsx', index=False)

# Simpan subset pengujian ke dalam file xlsx
test_data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/GetContact_DataTestEnglish.xlsx', index=False)


In [41]:
!pip install Textblob

In [42]:
from textblob import TextBlob

In [43]:
train_data

,content_normalisasi2
0,Good
1,solid
2,no minimum balance
3,limit
4,urgent
...,...
1717,nice to get flowers
1718,hopefully developing the application will be s...
1719,sure to make things easier
1720,yup


In [44]:
data_content = list(train_data['content_normalisasi2'])
polaritas = 0

status = []
total_positif = total_negatif = total = 0

for i, tweet in enumerate(data_content):
    analysis = TextBlob(tweet)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')

Hasil Analisis Data:
Positif = 1106
Negatif = 616

Total Data : 1722


In [45]:
status = pd.DataFrame({'Label': status})
train_data['Label'] = status
train_data

<ipython-input-45-ddb1f5b9bf00>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Label'] = status


,content_normalisasi2,Label
0,Good,Positif
1,solid,Negatif
2,no minimum balance,Negatif
3,limit,Negatif
4,urgent,Negatif
...,...,...
1717,nice to get flowers,Positif
1718,hopefully developing the application will be s...,Positif
1719,sure to make things easier,Positif
1720,yup,Negatif


In [46]:
# Memisahkan data berdasarkan label
positif_data = train_data[train_data['Label'] == 'Positif']
negatif_data = train_data[train_data['Label'] == 'Negatif']

In [47]:

# Menentukan jumlah sampel minimum di 2 kelas kelas
min_samples = min(total_positif, total_negatif)

In [48]:
from sklearn.utils import resample

In [49]:
# Melakukan penyeimbangan data dengan undersampling
positif_data_resampled = resample(positif_data, replace=False, n_samples=min_samples, random_state=42)
negatif_data_resampled = resample(negatif_data, replace=False, n_samples=min_samples, random_state=42)

In [50]:
# Menggabungkan kembali data yang telah disusun ulang
balanced_data = pd.concat([positif_data_resampled, negatif_data_resampled])

In [51]:
data_content_balanced = list(balanced_data['content_normalisasi2'])
polaritas_balanced = 0

status_balanced = []
total_positif_balanced = total_negatif_balanced = total_balanced = 0

for i, tweet in enumerate(data_content_balanced):
    analysis = TextBlob(tweet)
    polaritas_balanced += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif_balanced += 1
        status_balanced.append('Positif')
    else:
        total_negatif_balanced += 1
        status_balanced.append('Negatif')

    total_balanced += 1

print(f'Hasil Analisis Data Setelah Penyeimbangan:\nPositif = {total_positif_balanced}\nNegatif = {total_negatif_balanced}')
print(f'\nTotal Data Setelah Penyeimbangan: {total_balanced}')

Hasil Analisis Data Setelah Penyeimbangan:
Positif = 616
Negatif = 616

Total Data Setelah Penyeimbangan: 1232


In [52]:
train_data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/Seabank_DataTrainEngLabeled.xlsx',index=False)

In [53]:
balanced_data.to_excel('/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/Seabank_DataTrainEngLabeledBalanced.xlsx', index=False)

In [54]:
import pandas as pd
from googletrans import Translator

def translate_to_indonesian(text):
    translator = Translator()
    result = translator.translate(text, src='en', dest='id')
    return result.text

def translate_excel_column(input_file, output_file, sheet_name, column_name):
    df = pd.read_excel(input_file, sheet_name=sheet_name)
    df[column_name] = df[column_name].apply(translate_to_indonesian)

    with pd.ExcelWriter(output_file) as writer:
        df.to_excel(writer, index=False, sheet_name=sheet_name)
    print("Translation complete. Translated data saved to a new Excel file!")

# Example usage
input_file = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/Seabank_DataTrainEngLabeledBalanced.xlsx'
output_file = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/Seabank_DataTrainIdnLabeledBalanced.xlsx'
sheet_name = 'Sheet1'  # Change to your sheet name
column_name = 'content_normalisasi2'  # Change to the name of the column you want to translate

translate_excel_column(input_file, output_file, sheet_name, column_name)

Translation complete. Translated data saved to a new Excel file!


In [57]:
import pandas as pd
from googletrans import Translator

def translate_to_indonesian(text):
    translator = Translator()
    result = translator.translate(text, src='en', dest='id')
    return result.text

def translate_excel_column(input_file, output_file, sheet_name, column_name):
    df = pd.read_excel(input_file, sheet_name=sheet_name)
    df[column_name] = df[column_name].apply(translate_to_indonesian)

    with pd.ExcelWriter(output_file) as writer:
        df.to_excel(writer, index=False, sheet_name=sheet_name)
    print("Translation complete. Translated data saved to a new Excel file!")

# Example usage
input_file = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/GetContact_DataTestEnglish.xlsx'
output_file = '/content/drive/MyDrive/Sentimen Analisis 3 Top Bank Digital/Seabank_DataTestIndo.xlsx'
sheet_name = 'Sheet1'  # Change to your sheet name
column_name = 'content_normalisasi2'  # Change to the name of the column you want to translate

translate_excel_column(input_file, output_file, sheet_name, column_name)

Translation complete. Translated data saved to a new Excel file!
